# Tekla PowerFab Database Queries

This notebook connects to the Tekla PowerFab MySQL database and provides sample queries for extracting data.

## Setup

Before running this notebook:
1. Install dependencies: `pip install -r requirements.txt`
2. Copy `.env.example` to `.env` and update credentials
3. Ensure the MySQL service is running (check Windows Services for `MySQL_TeklaPowerFab`)

## 1. Import Libraries and Setup Connection

In [2]:
import mysql.connector
import pandas as pd
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Database connection settings
DB_CONFIG = {
    'host': os.getenv('MYSQL_HOST', 'localhost'),
    'port': int(os.getenv('MYSQL_PORT', 3307)),
    'user': os.getenv('MYSQL_USER', 'admin'),
    'password': os.getenv('MYSQL_PASSWORD', '^Fymx^r4'),
    'database': os.getenv('MYSQL_DATABASE', 'all-things-metal'),
    'use_pure': True,  # Use pure Python implementation (fixes auth plugin issues)
    'auth_plugin': 'mysql_native_password'  # Explicitly specify auth plugin
}

print(f"Connecting to: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")

Connecting to: localhost:3307/fabrication


## 2. Helper Function for Running Queries

In [3]:
def run_query(query, params=None):
    """
    Execute a SQL query and return results as a pandas DataFrame.
    
    Args:
        query: SQL query string
        params: Optional tuple of parameters for parameterized queries
    
    Returns:
        pandas DataFrame with query results
    """
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        df = pd.read_sql(query, conn, params=params)
        conn.close()
        return df
    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return None

def get_connection():
    """
    Get a database connection for manual operations.
    Remember to close the connection when done.
    """
    return mysql.connector.connect(**DB_CONFIG)

## 3. Test Connection

In [4]:
# Test the connection
try:
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT VERSION()")
    version = cursor.fetchone()
    print(f"Connected successfully! MySQL version: {version[0]}")
    cursor.close()
    conn.close()
except mysql.connector.Error as err:
    print(f"Connection failed: {err}")
    print("\nTroubleshooting:")
    print("1. Check if MySQL_TeklaPowerFab service is running in Windows Services")
    print("2. Verify credentials in .env file")
    print("3. Ensure port 3306 is not blocked")

Connected successfully! MySQL version: 8.0.35


## 4. List All Tables in Database

In [5]:
# Get all tables in the fabrication database
tables_df = run_query("SHOW TABLES")
if tables_df is not None:
    print(f"Total tables: {len(tables_df)}")
    display(tables_df)

Total tables: 1285


C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,Tables_in_fabrication
0,accessoryitemestimateextras
1,accessoryitemlinks
2,accessoryitems
3,accessoryitemslog
4,advancebillpendingchanges
...,...
1280,workpackagedepartmenttaskslog
1281,workpackages
1282,workpackageslog
1283,workshops


## 5. Explore Table Structure

Use this to see the columns in any table.

In [6]:
# Change this to explore different tables
TABLE_NAME = 'assemblies'

structure_df = run_query(f"DESCRIBE {TABLE_NAME}")
if structure_df is not None:
    print(f"Structure of '{TABLE_NAME}' table:")
    display(structure_df)

Structure of 'assemblies' table:


C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,Field,Type,Null,Key,Default,Extra
0,AssemblyID,mediumint,NO,PRI,None,auto_increment
1,Description,varchar(255),YES,,None,
2,EstimateID,bigint,YES,,None,


## 6. Sample Queries

Below are sample queries for common PowerFab data. Modify as needed.

### 6.1 Get Sample Data from Assemblies

In [7]:
# Get first 100 rows from assemblies table
assemblies_df = run_query("SELECT * FROM assemblies LIMIT 100")
if assemblies_df is not None:
    print(f"Assemblies: {len(assemblies_df)} rows")
    display(assemblies_df.head(20))

Assemblies: 38 rows


C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,AssemblyID,Description,EstimateID
0,1,Elevator Pit Ladder,NaN
1,2,Frame,NaN
2,3,Trash Gate,114.0
3,4,Stiffner,NaN
4,5,Wall Mt Bracket,NaN
5,6,Ladder Cage,NaN
6,7,"Stiffener 10""",NaN
7,8,"Stiffener 12""",NaN
8,9,"Stiffener 14""",NaN
9,10,"Stiffener 16""",NaN


### 6.2 Get Jobs/Projects Data

In [26]:
# Query for jobs - adjust table name if different in your database
jobs_query = """
SHOW TABLES LIKE '%track%'
"""
jobs_tables = run_query(jobs_query)
if jobs_tables is not None:
    print("Tables containing 'job':")
    display(jobs_tables)

Tables containing 'job':


C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,Tables_in_fabrication (%track%)
0,tempproductioncontrolitemsequenceloadtracking
1,tempproductioncontrolloadtracking
2,tempproductioncontrolloadtrackingload
3,tempproductioncontrolloadtrackingreturn
4,tempproductioncontrolpiecetracking


### 6.3 Search for Tables by Keyword

In [27]:
# Search for tables containing a keyword
KEYWORD = 'track'  # Change this to search for different tables

search_query = f"SHOW TABLES LIKE '%{KEYWORD}%'"
matching_tables = run_query(search_query)
if matching_tables is not None:
    print(f"Tables containing '{KEYWORD}':")
    display(matching_tables)

Tables containing 'track':


C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,Tables_in_fabrication (%track%)
0,tempproductioncontrolitemsequenceloadtracking
1,tempproductioncontrolloadtracking
2,tempproductioncontrolloadtrackingload
3,tempproductioncontrolloadtrackingreturn
4,tempproductioncontrolpiecetracking


In [10]:
search_query = f"DESCRIBE timerecords"
run_query(search_query)

C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,Field,Type,Null,Key,Default,Extra
0,TimeRecordID,int unsigned,NO,PRI,None,auto_increment
1,ProjectID,bigint,YES,MUL,None,
2,EmployeeUserID,bigint,YES,MUL,None,
3,RecordUnixTime,int unsigned,YES,MUL,None,
4,StartDate,date,YES,,None,
5,StartUnixTime,int unsigned,YES,,None,
6,EndUnixTime,int unsigned,YES,,None,
7,RegularHours,"decimal(35,15)",NO,,None,
8,OvertimeHours,"decimal(35,15)",NO,,None,
9,Overtime2Hours,"decimal(35,15)",NO,,None,


In [11]:
search_query = f"DESCRIBE timerecordsubjects"
run_query(search_query)

C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,Field,Type,Null,Key,Default,Extra
0,TimeRecordSubjectID,int unsigned,NO,PRI,None,auto_increment
1,SubjectHash,binary(20),NO,UNI,None,


In [12]:
search_query = f"DESCRIBE timerecordsubjects"
run_query(search_query)

C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,Field,Type,Null,Key,Default,Extra
0,TimeRecordSubjectID,int unsigned,NO,PRI,None,auto_increment
1,SubjectHash,binary(20),NO,UNI,None,


In [13]:
search_query = f"DESCRIBE scheduletasktimerecords"
run_query(search_query)

C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,Field,Type,Null,Key,Default,Extra
0,ScheduleTaskID,int unsigned,NO,PRI,None,
1,TimeRecordID,int unsigned,NO,PRI,None,


In [14]:
search_query = f"SHOW TABLES LIKE '%employee%'"
run_query(search_query)

C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,Tables_in_fabrication (%employee%)


In [15]:
search_query = f"SHOW TABLES LIKE '%project%'"
run_query(search_query)

C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,Tables_in_fabrication (%project%)
0,externalprojects
1,externalprojectslog
2,externalprojectsync
3,projectbudgetinvoiceitems
4,projectbudgetinvoiceitemslog
5,projectbudgetinvoices
6,projectbudgetinvoiceslog
7,projectbudgetitems
8,projectbudgetitemslog
9,projectbudgets


In [16]:
search_query = f"SHOW TABLES LIKE '%station%'"
run_query(search_query)

C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,Tables_in_fabrication (%station%)
0,productioncontrolitemstationinstancenumbers
1,productioncontrolitemstations
2,productioncontrolitemstationslog
3,productioncontrolitemstationsummary
4,productioncontrolitemstationsummaryinstancenum...
5,productioncontrolitemstationsummarylog
6,routestations
7,routestationslog
8,stationlaborgroups
9,stationlaborgroupslog


In [17]:
run_query("SHOW TABLES LIKE '%user%'")

C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,Tables_in_fabrication (%user%)
0,tempuserpermissions
1,useraccess
2,userfilteritems
3,userfilters
4,userformlayout
5,userformstate
6,userpermissiondetails
7,userpermissions
8,userpreferences
9,userpreferenceslog


In [18]:
run_query("DESCRIBE users")

C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,Field,Type,Null,Key,Default,Extra
0,UserID,bigint,NO,PRI,None,auto_increment
1,Username,varchar(255),YES,MUL,None,
2,LastName,varchar(255),YES,,None,
3,FirstName,varchar(255),YES,,None,
4,UserGroup,varchar(255),YES,,None,
5,TrimbleIdentityAccount,varchar(255),YES,,None,
6,LastTrimbleIdentityLogin,int unsigned,YES,,None,
7,HasAdministrativePermission,tinyint(1),YES,,None,
8,UserRoleID,bigint,YES,,None,
9,ExternalUser,tinyint(1),YES,,None,


In [ ]:
run_query("""
      SELECT
          tr.TimeRecordID,
          tr.ProjectID,
          tr.EmployeeUserID,
          tr.StartDate,
          tr.RegularHours,
          tr.OvertimeHours,
          tr.TimeRecordSubjectID,
          tr.InProgress
      FROM timerecords tr
      LIMIT 10
  """)

C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,TimeRecordID,ProjectID,EmployeeUserID,StartDate,RegularHours,OvertimeHours,TimeRecordSubjectID,InProgress
0,1,4,7.0,2022-08-30,2.00,1.0,1,0
1,2,4,10.0,2022-08-30,2.00,1.0,1,0
2,3,61,NaN,2022-12-07,100.00,0.0,2,0
3,4,61,31.0,2022-12-08,0.10,0.0,1,0
4,5,34,12.0,2022-12-08,0.17,0.0,2,0
5,6,61,12.0,2022-12-08,0.12,0.0,3,0
6,7,61,12.0,2022-12-08,0.02,0.0,3,0
7,19,27,42.0,2023-02-15,0.52,0.0,12,0
8,20,27,42.0,2023-02-15,1.73,0.0,13,0
9,21,27,42.0,2023-02-15,1.37,0.0,14,0


In [22]:
run_query("SELECT * FROM projects LIMIT 1")

C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,ProjectID,JobNumber,JobDescription,JobLocation,JobStatusID,JobDate,CustomerPONumber,DrawingNumberInputTypeID,DrawingFilePrefix,DrawingFileSuffix,UpdateProductionControlApprovalStatus,GroupName,GroupName2,LastReleaseNumber,CurrentScheduleBaselineID,ExternalProjectID,ProjectSightExternalProjectID,ERPJobNumber,CostCenter
0,6,21024,Raintree Apartments,"8555 E. Raintree Dr., Scottsdale, Arizona 85260",2,2022-06-02,SC-S122500-010,13,None,None,1,Derek,Apartments,29.0,6,69,None,21024,None


In [24]:
run_query("SELECT * FROM stations LIMIT 10")

C:\Users\nickb\AppData\Local\Temp\ipykernel_33172\630592994.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


,StationID,StationNumber,StationType,Description,CostCodeID,CostTypeID,DepartmentID,XCoord,YCoord,Width,Height,Size
0,1,10,0,1-Cut/Saw,None,None,1.0,30,90,50,50,Medium
1,2,20,0,QC Fitup Ins,None,None,1.0,120,90,50,50,Medium
2,3,30,0,Final QC,None,None,1.0,200,90,50,50,Medium
3,6,50,1,a-On site,None,None,NaN,0,0,30,30,None
4,7,60,1,b-Erected,None,None,NaN,30,0,30,30,None
5,8,70,0,Material Handling,None,None,1.0,0,350,30,30,None
6,9,71,0,Rework Own,None,None,1.0,30,350,30,30,None
7,10,72,0,Rework Others,None,None,1.0,60,350,30,30,None
8,11,73,0,Time n Material,None,None,1.0,90,350,30,30,None
9,12,74,0,Paint/Primer,None,None,1.0,0,30,30,30,None


## 7. Custom Query

Use this cell to write your own queries.

In [10]:
# Write your custom query here
custom_query = """
SELECT * 
FROM assemblies 
LIMIT 10
"""

result = run_query(custom_query)
if result is not None:
    display(result)

Database error: 1045 (28000): Access denied for user 'RickyH'@'127.0.0.1' (using password: YES)


## 8. Export Data to CSV

In [11]:
# Export query results to CSV
export_query = """
SELECT * FROM assemblies LIMIT 1000
"""

export_df = run_query(export_query)
if export_df is not None:
    output_file = 'exported_data.csv'
    export_df.to_csv(output_file, index=False)
    print(f"Data exported to {output_file}")
    print(f"Rows exported: {len(export_df)}")

Database error: 1045 (28000): Access denied for user 'RickyH'@'127.0.0.1' (using password: YES)
